In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import lsst.daf.butler as dafButler

plt.set_loglevel('WARNING')

INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
# Load collection
repo = '/repo/main'
instrument = "LSSTComCam"
butler= dafButler.Butler(repo, collections='u/bos/DM-48672/LSSTComCam_drp_association_v2', instrument=instrument)
butler_w04= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_04/DM-48556', instrument=instrument)

In [6]:
lc_refs = butler.query_datasets("goodSeeingDiff_assocDiaSrcTable")
lc_refs_04 = butler_w04.query_datasets("goodSeeingDiff_assocDiaSrcTable")

print(f"Number of tracts in diaSourceTable (current): {len(lc_refs)}")
print(f"Number of tracts in diaSourceTable (current): {len(lc_refs_04)}")


Number of tracts in diaSourceTable (current): 587
Number of tracts in diaSourceTable (current): 586


In [ ]:
# List of patch indices to process
patch_indices = np.arange(0, len(lc_refs))

diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tqdm(patch_indices):
    obj_ref = lc_refs[idx]  # Select the corresponding obj_ref
    diaSource_patch = butler.get('goodSeeingDiff_assocDiaSrcTable', dataId=obj_ref.dataId)  # Fetch diaSource for this patch
    diaSource_list.append(diaSource_patch)

diaSource = pd.concat(diaSource_list, ignore_index=False)

  0%|▏                                                                                                                                                 | 1/587 [00:00<03:26,  2.83it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 587/587 [04:10<00:00,  2.34it/s]


In [8]:
# List of patch indices to process
patch_indices = np.arange(0, len(lc_refs_04))

diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tqdm(patch_indices):
    obj_ref = lc_refs_04[idx]  # Select the corresponding obj_ref
    diaSource_patch = butler_w04.get('goodSeeingDiff_assocDiaSrcTable', dataId=obj_ref.dataId)  # Fetch diaSource for this patch
    diaSource_list.append(diaSource_patch)


diaSource_04 = pd.concat(diaSource_list, ignore_index=False)


  0%|                                                                                                                                                          | 0/586 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 586/586 [01:16<00:00,  7.70it/s]


In [9]:
# Count the number of unique diaObjectId values for each diaSourceId
object_count_per_source = diaSource.groupby("diaSourceId")["diaObjectId"].nunique()

# Convert to DataFrame for easier handling
object_count_df = object_count_per_source.reset_index()
object_count_df.columns = ["diaSourceId", "num_unique_diaObjectId"]


# Count the number of unique diaObjectId values for each diaSourceId
object_count_per_source_04 = diaSource_04.groupby("diaSourceId")["diaObjectId"].nunique()

# Convert to DataFrame for easier handling
object_count_df_04 = object_count_per_source_04.reset_index()
object_count_df_04.columns = ["diaSourceId", "num_unique_diaObjectId"]


In [10]:
# diaSources that are associated with more than one diaObject in the new version
object_count_df[object_count_df['num_unique_diaObjectId']>1]

,diaSourceId,num_unique_diaObjectId
1,190875481648660482,2
4,190875481648660485,3
5,190875481648660486,3
7,190875481648660488,2
10,190875481648660491,3
...,...,...
5083549,192036958950655465,2
5083550,192036958950655466,2
5083551,192036958950655467,2
5083552,192036958950655468,2


In [12]:
# diaSources that are associated with more than one diaObject in the weekly 04
object_count_df_04[object_count_df_04['num_unique_diaObjectId']>1]

,diaSourceId,num_unique_diaObjectId
1,190875481648660482,2
4,190875481648660485,3
5,190875481648660486,3
7,190875481648660488,2
10,190875481648660491,3
...,...,...
5083565,192036958950655465,2
5083566,192036958950655466,2
5083567,192036958950655467,2
5083568,192036958950655468,2
